使用Mido第三方库，在展开实验前，先规定一些参数

In [ ]:
import mido
import numpy as np
from random import randint, choice, random
from typing import List, Literal, Union
from copy import deepcopy
import math

BPM = 120  # Beats per minute
VELOCITY = 64  # MIDI note velocity
TICK_PER_BEAT = 480  # Tick per beat

# The rhythm of the generated music
NUMERATOR = 4
DENOMINATOR = 4  # 4/4

# The range of notes to be generated
NOTE_MIN = 60  # C4
NOTE_MAX = 84  # C6

# Note lengths
WHOLE = 4 * TICK_PER_BEAT
HALF = 2 * TICK_PER_BEAT
QUARTER = 1 * TICK_PER_BEAT
EIGHTH = QUARTER // 2
SIXTEENTH = QUARTER // 4
NOTE_LENGTH = [WHOLE, HALF, QUARTER, EIGHTH]

BAR_LENGTH = NUMERATOR * TICK_PER_BEAT

音符的包装——包含起始时间、持续时间、音高、音量的属性，同时提供了音名翻译的接口。

In [ ]:
note_name_dict = {
    'C': 0, 'C#': 1, 'Cb': -1,
    'D': 2, 'D#': 3, 'Db': 1,
    'E': 4, 'E#': 5, 'Eb': 3, 
    'F': 5, 'F#': 6, 'Fb': 4,
    'G': 7, 'G#': 8, 'Gb': 6, 
    'A': 9, 'A#': 10, 'Ab': 8,
    'B': 11, 'B#': 12, 'Bb': 10,
}

Key_Major_T = Literal['C', 'Db', 'D', 'Eb', 'E', 'F',
                      'F#', 'E', 'G', 'Ab', 'A', 'Bb', 'B']
Key_Minor_T = Literal['Cm', 'C#m', 'Dm', 'D#m', 'Ebm', 'Em', 
                      'Fm', 'F#m', 'Gm', 'G#m', 'Am', 'Bbm', 'Bm']
Key_T = Union[Key_Major_T, Key_Minor_T]
Pitch_T = int

# offset of the notes in the major mode and minor mode
major_offset = (0, 2, 4, 5, 7, 9, 11)
minor_offset = (0, 2, 3, 5, 7, 8, 10)


class Note:
    def __init__(self, pitch: Pitch_T, length: int, 
                 start_time: int, velocity: int = VELOCITY):
        # Here the "time" is "tick" in mido actually
        self.pitch = pitch
        self.length = length
        self.start_time = start_time
        self.velocity = velocity

    def __str__(self):
        pitch = self.pitch_to_name(self.pitch)
        return f"{pitch}: length={self.length} start={self.start_time} velocity={self.velocity}"

    @staticmethod
    def in_mode(pitch: Pitch_T, key: Key_T):
        """judge if the note is in the given mode."""
        if key.endswith('m'):
            # minor mode
            base = note_name_dict[key[:-1]]
            return (pitch - base) % 12 in minor_offset
        else:
            # major mode
            base = note_name_dict[key]
            return (pitch - base) % 12 in major_offset
    
    @staticmethod
    def random_pitch_in_mode(key: Key_T):
        """Generate a random note in the given mode."""
        while True:
            pitch = randint(NOTE_MIN, NOTE_MAX)
            if Note.in_mode(pitch, key):
                return pitch
            
    @staticmethod
    def name_to_pitch(note_name: str) -> Pitch_T:
        """Convert a note name to a pitch.
        For example, "C4" -> 60. """
        octave = note_name[-1]
        name = note_name[:-1]
        pitch = note_name_dict[name]
        return (int(octave) + 1) * 12 + pitch

    @staticmethod
    def pitch_to_name(pitch: Pitch_T) -> str:
        """Convert a pitch to a note name.
        For example, 60 -> "C4"."""
        octave = pitch // 12 - 1
        note = pitch % 12
        for name, picth in note_name_dict.items():
            if note == picth:
                return name + str(octave)

对音轨的包装
- `self.note`是所有音符构成的列表，直接访问即可
- 提供了与`mido.MidiTrack`相互转化的接口
- 完成了对音轨的移调、倒影、逆行工作

In [ ]:
class Track:
    """A wrapper for mido.MidiTrack."""

    def __init__(self, instrument: int = 0, key: Key_T = 'C'):
        self.instrument = instrument
        self.key = key
        self.note: list[Note] = []

    def __str__(self):  # used for debug
        meta_msg = f"Key: {self.key}\nInstrument: {self.instrument}\n"
        note_msg = '\n'.join([str(note) for note in self.note])
        return meta_msg + note_msg

    def print_brief_info(self):
        """Brief information of the track."""
        print(f"Key: {self.key}")
        print(f"Instrument: {self.instrument}")
        print(f"Length: {self.full_length}")
        print(f"Bar: {self.bar_number}")

    @staticmethod
    def from_track(track: mido.MidiTrack):
        """Generate a track from a midi track.
        Available for chords."""

        ga_track = Track()
        time = 0
        note_dict = {}  # pitch -> (start_time, velocity)
        for msg in track:
            if msg.type == 'program_change':
                ga_track.instrument = msg.program
            elif msg.type == 'key_signature':
                ga_track.key = msg.key
            elif msg.type == 'note_on':
                time += msg.time
                note_dict[msg.note] = (time, msg.velocity)
            elif msg.type == 'note_off':
                time += msg.time
                start_time, velocity = note_dict.pop(msg.note)
                ga_track.note.append(
                    Note(msg.note, time - start_time, start_time, velocity))
        return ga_track

    def to_track(self) -> mido.MidiTrack:
        """Generate a midi track from a track.
        Available for chords."""

        midi_track = mido.MidiTrack()
        midi_track.append(mido.MetaMessage(
            'key_signature', key=self.key, time=0))
        midi_track.append(mido.Message(
            'program_change', program=self.instrument, time=0))

        event_set = set()  # (time, 0/1(note_off/on), note)
        for note in self.note:
            event_set.add((note.start_time, 1, note))
            event_set.add((note.start_time + note.length, 0, note))
        # Sort all the events by time. If the time is same, note_off is before note_on.
        sorted_time = sorted(event_set, key=lambda x: (x[0], x[1]))

        last_time = 0
        for time, event_num, note in sorted_time:
            event = 'note_on' if event_num else 'note_off'
            msg = mido.Message(
                event, note=note.pitch, velocity=note.velocity, time=time-last_time)
            midi_track.append(msg)
            last_time = time
        return midi_track

    def generate_random_track(self, bar: int):
        """Generate a random track with the given bar length."""
        for i in range(bar):
            length = BAR_LENGTH
            while length > 0:
                l = choice(NOTE_LENGTH)
                if l <= length:
                    pitch = Note.random_pitch_in_mode(self.key)
                    note = Note(pitch, l, (i + 1) * BAR_LENGTH - length)
                    length -= l
                    self.note.append(note)
        return self

    def split_into_bars(self) -> List[List[Note]]:
        """Split the track into bars."""
        bars = [[] for _ in range(self.bar_number)]
        for note in self.note:
            idx = note.start_time // BAR_LENGTH

            if note.start_time + note.length <= (idx + 1) * BAR_LENGTH:
                bars[idx].append(note)
            else:
                # The note exceeds the bar, split it into two parts
                note1, note2 = deepcopy(note), deepcopy(note)
                bar_time = (idx + 1) * BAR_LENGTH
                note1.length = bar_time - note.start_time
                note2.length = note.start_time + note.length - bar_time
                note2.start_time = bar_time
                bars[idx].append(note1)
                bars[idx + 1].append(note2)
        return bars
    
    def join_bars(self, bars: List[List[Note]]):
        """Join the bars into a track."""
        self.note = [note for bar in bars for note in bar]
        return self
    
    @property
    def full_length(self):
        """The length of the track"""
        return max(note.start_time + note.length for note in self.note)

    @property
    def bar_number(self):
        """The number of bars"""
        return math.ceil(self.full_length / WHOLE)

    def transpose(self, interval):
        """Transpose the track by the given interval"""
        for note in self.note:
            note.pitch += interval

    def inverse(self, center):
        """Inverse the track by the given center"""
        for note in self.note:
            note.pitch = 2 * center - note.pitch

    def retrograde(self):
        """Retrograde the track"""
        for note in self.note:
            note.start_time = self.full_length - note.start_time - note.length
        self.note.reverse()

测试函数
- `get_midi` 生成一个给定调式的空白midi，同时建立好节拍、曲速、调式的元信息
- `generate_random_midi` 生成一个随机midi，调式指定为#g小调
- `read_midi` 读取一个现有的midi——久石让的《Summer》间奏片段。其中包含和弦、音符强弱变化以用于测试

In [ ]:
def get_midi(key: Key_T = None):
    s = mido.MidiFile()
    meta_track = mido.MidiTrack()
    meta_track.append(mido.MetaMessage('set_tempo', tempo=mido.bpm2tempo(BPM)))
    meta_track.append(mido.MetaMessage(
        'time_signature', numerator=NUMERATOR, denominator=DENOMINATOR))
    if key is not None:
        meta_track.append(mido.MetaMessage(
            'key_signature', key=key, time=0))
    s.tracks.append(meta_track)
    return s

In [ ]:
def generate_random_midi_test():
    s = get_midi()
    track = Track(0, 'G#m')
    track.generate_random_track(4)  # 4 bars
    s.tracks.append(track.to_track())
    retrograde_track = deepcopy(track)  # MUST use deepcopy
    retrograde_track.retrograde()
    s.tracks.append(retrograde_track.to_track())
    s.save('random.mid')

    # Expected result:
    # random.mid: A random piece with 4 bars in G sharp minor which has 2 tracks,
    # one is the original track, the other is the retrograde track.


def read_midi_test():
    s = mido.MidiFile('test.mid')
    right_hand = Track.from_track(s.tracks[0])
    right_hand.print_brief_info()
    print('----------------')
    left_hand = Track.from_track(s.tracks[1])
    left_hand.print_brief_info()
    # For more information, use print(track) to see the detailed notes.
    print(left_hand.bar_number)

    # Expected output:
    # Key: D
    # Instrument: 0
    # Length: 15353
    # Bar: 8
    # ----------------
    # Key: D
    # Instrument: 0
    # Length: 15347
    # Bar: 8


generate_random_midi_test()
read_midi_test()

接下来做遗传算法的准备工作

In [ ]:
# interval -> value
# 0 fot unison, 12 for octave
# if interval > 12, the value will be 3
interval_value_dict = {
    0: 1, 1: 3, 2: 3, 3: 2, 4: 2, 5: 1, 6: 3, 7: 1, 8: 2, 9: 2, 10: 3, 11: 3, 12: 1
}


class TrackParameter:
    """Used to calculate parameters of the tracks"""

    def __init__(self, track: Track) -> None:
        self.track = track
        self.bar_number = track.bar_number
        self.means = np.zeros(self.bar_number, dtype=float)
        self.variances = np.zeros(self.bar_number, dtype=float)
        self.bad_notes = 0
        self.update_parameters()

    @staticmethod
    def _interval_to_value(interval: int):
        return interval_value_dict.get(abs(interval), 3)

    def update_parameters(self):
        """Return the means and variances of the values of the track"""
        bars = self.track.split_into_bars()
        self.bad_notes = 0

        for idx, bar in enumerate(bars):
            values = []
            for i in range(len(bar) - 1):
                interval = bar[i + 1].pitch - bar[i].pitch
                values.append(self._interval_to_value(interval))
            if idx > 0:
                # add the interval between the last note of the previous bar
                # and the first note of the current bar
                interval = bar[0].pitch - bars[idx - 1][-1].pitch
                values.append(self._interval_to_value(interval))

            if values == []:
                self.means[idx] = 0
                self.variances[idx] = 0
                continue

            self.means[idx] = np.mean(values)
            self.variances[idx] = np.var(values)

            for note in bar:
                if not note.in_mode(note.pitch, self.track.key):
                    self.bad_notes += 1

In [ ]:
alpha, beta, gamma = 1, 1, 0.5
mean_coeff = np.array([2, 1, 1, 1, 1, 1, 1, 2])
varience_coeff = np.array([1, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 1])
target = 1.8

class TrackPopulation:

    def __init__(self,
                 reference_track: Track,
                 population: List[Track],
                 mutation_rate: float = 0.1
                 ):

        self.ref_track = reference_track
        self.ref_param = TrackParameter(self.ref_track)
        self.population = population
        self.mutation_rate = mutation_rate
        self.fitness = [0] * len(population)
        self._update_fitness()
        self.best_index, self.second_index = 0, 0

    def _update_fitness(self):
        for idx, track in enumerate(self.population):
            self.fitness[idx] = self._get_fitness(track)

    def _get_fitness(self, track: Track)->float:
        # It's better to have a lower fitness
        track_param = TrackParameter(track)
        mean_diff = np.abs(track_param.means - self.ref_param.means)
        f1 = alpha * np.dot(mean_coeff, mean_diff)
        varience_diff = np.abs(track_param.variances - self.ref_param.variances)
        f2 = beta * np.dot(varience_coeff, varience_diff)
        
        g = gamma * TrackParameter(track).bad_notes
        return f1 + f2 + g

    def select(self):
        self._update_fitness()
        for i in range(len(self.fitness)):
            if self.fitness[i] < self.fitness[self.best_index]:
                self.best_index = i
                self.second_index = self.best_index
            elif self.fitness[i] < self.fitness[self.second_index]:
                self.second_index = i

    def crossover(self):
        # for i in range(len(self.population)):
        #     index1 = self.best_index if randint(0,1) else self.second_index
        #     index2 = self.best_index if randint(0,1) else self.second_index
        #     bars1 = deepcopy(self.population[index1]).split_into_bars()
        #     bars2 = deepcopy(self.population[index2]).split_into_bars()
        #     cross_point = randint(0, self.ref_param.bar_number - 1)
        #     bars = bars1[:cross_point] + bars2[cross_point:]
        #     self.population[i] = self.population[i].join_bars(bars)
        # self._update_fitness()
        pass

    def mutate(self):
        for i in range(len(self.population)):
            if random() > self.mutation_rate:
                continue
            # TODO: mutation
            track = deepcopy(self.population[choice([self.best_index,self.second_index])])
            mutate_type = randint(1,3)
            if mutate_type == 1:
                self._mutate_1(track)
            elif mutate_type == 2:
                self._mutate_2(track)
            else:
                self._mutate_3(track)
            self.population[i] = track

    def _mutate_1(self, track: Track):
        for idx in range(1, len(track.note)):
            if track.note[idx].pitch - track.note[idx - 1].pitch > 12:
                track.note[idx].pitch -= 12
            elif track.note[idx].pitch - track.note[idx - 1].pitch < -12:
                track.note[idx].pitch += 12
                
    def _mutate_2(self, track: Track):
        idx = randint(0, len(track.note) - 1)
        track.note[idx].pitch = Note.random_pitch_in_mode(track.key)
        
    def _mutate_3(self, track: Track): 
        idx = randint(1, len(track.note) - 1)
        track.note[idx], track.note[idx - 1] = track.note[idx - 1], track.note[idx]

    def show_info(self):
        print("Now the best fitness is", self.fitness[self.best_index])

    def run(self, generation):
        for i in range(generation):
            print(f"<epoch {i} begins>")
            self.select()
            self.crossover()
            self.mutate()
            self.show_info()
            print(f"<epoch {i} ends>")
            if self.fitness[self.best_index] < target:
                print(f"[!] Target reached at epoch {i}")
                print(f"final fitness: {self.fitness[self.best_index]}")
                return self.population[self.best_index]
            
        print(f"[!] Target not reached after {generation} generations")
        print(f"final fitness: {self.fitness[self.best_index]}")
        return self.population[self.best_index]

遗传算法的运行测试

In [ ]:
from time import time

KEY = 'E'
t0 = time()

ref_midi = mido.MidiFile('reference.mid')
ref_track = Track.from_track(ref_midi.tracks[0])

population = [Track(0, KEY).generate_random_track(8) for _ in range(10)]
ga = TrackPopulation(ref_track, population, 0.8)
result = ga.run(1000)

s = get_midi(KEY)
s.tracks.append(result.to_track())
s.save('result.mid')

print(f"Time cost: {time() - t0}s")
